In [ ]:
# from google.colab import drive
# import shutil
# drive.mount("/content/drive")

In [ ]:
! pip install ultralytics==8.0.196
! pip install cv2


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement cv2 (from versions: none)
ERROR: No matching distribution found for cv2

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from ultralytics import YOLO

In [ ]:
# !pip install roboflow

# from roboflow import Roboflow
# rf = Roboflow(api_key="vPTtZp7Wh7P8lRdRPdGe")
# project = rf.workspace("cs-1430").project("wr-finder")
# version = project.version(2)
# dataset = version.download("yolov8")


### Fine Tuning

In [ ]:
# Load the model.
model = YOLO('yolov8n.pt')

# Training.
results = model.train(
   data='NewWRData/data2.yaml',
   imgsz=640,
   epochs=10,
   batch=8,
   name='yolov8n_WR')

New https://pypi.org/project/ultralytics/8.2.11 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196  Python-3.11.0 torch-2.3.0+cpu CPU (Intel Core(TM) i7-9750H 2.60GHz)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=NewWRData/data2.yaml, epochs=10, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8n_WR, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False,

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:11<00:00,  2.76s/it]
                   all         60        781      0.429      0.544      0.403      0.194

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       7/10         0G      1.682      1.877      1.244         24        640: 100%|██████████| 27/27 [01:14<00:00,  2.77s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:11<00:00,  2.75s/it]
                   all         60        781      0.531      0.487      0.464      0.215

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       8/10         0G      1.676      1.793      1.258         26        640: 100%|██████████| 27/27 [01:14<00:00,  2.75s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:10<00:00,  2.71s/it]
  

In [ ]:
# src = "/content/runs"
# dst = "/content/drive/MyDrive/WR/runs"
# shutil.move(src, dst)

### Load model directly from trained weights

In [ ]:
best_model = YOLO("yolov8n_WR/weights/best.pt")

### Evaluate and Visualization

In [ ]:
import cv2
import numpy as np
# from google.colab.patches import cv2_imshow

In [ ]:
def non_max_suppression(boxes, scores, labels, threshold):
    # Initialize lists to store selected boxes and scores
    selected_boxes = []
    selected_scores = []
    selected_labels = []

    # Sort the boxes by their scores (in descending order)
    sorted_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)

    while len(sorted_indices) > 0:
        # Pick the box with the highest score
        i = sorted_indices[0]
        selected_boxes.append(boxes[i])
        selected_scores.append(scores[i])
        selected_labels.append(labels[i])

        # Calculate overlap with other boxes
        overlap = [calculate_overlap(boxes[i], boxes[j]) for j in sorted_indices[1:]]

        # Filter out boxes with overlap greater than the threshold
        filtered_indices = [j for j, o in zip(sorted_indices[1:], overlap) if o <= threshold]

        # Update sorted_indices to remove selected and redundant boxes
        sorted_indices = filtered_indices

    return selected_boxes, selected_scores, selected_labels

def calculate_overlap(box1, box2):
    # Calculate the intersection area
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    intersection_area = max(0, x2 - x1 + 1) * max(0, y2 - y1 + 1)

    # Calculate the union area
    area1 = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
    area2 = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)
    union_area = area1 + area2 - intersection_area

    # Calculate the overlap ratio
    overlap = intersection_area / union_area

    return overlap

In [ ]:
def plot_bounding_box(img, result, threshold=0.5):
  conf = result.boxes.conf.numpy()
  boxes = result.boxes.xyxy.numpy()
  labels = result.boxes.cls.numpy()
  selected_boxes, selected_scores, selected_labels = non_max_suppression(boxes, conf, labels, threshold)
  names = result.names
  for i in range(len(selected_boxes)):
    x1, y1, x2, y2 = selected_boxes[i]
    cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
    cv2.putText(img, names[labels[i]], (int(x1)-20, int(y1)-10), cv2.FONT_HERSHEY_PLAIN, 0.8, (0, 255, 0), 1)

In [ ]:
! pip install opencv-python --upgrade
img = cv2.imread("../WR/NewWRData/test/images/SuperBowl32_png.rf.21051910af3614876b4bb97ea537edbe.jpg")
results = best_model.predict(img)
plot_bounding_box(img, results[0])
# window_name = 'image'
# cv2.imshow(window_name, img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

0: 640x640 2 Corner Backs, 2 Linebackers, 1 Running Back, 2 Safetys, 3 Wide Receivers, 171.8ms


Speed: 3.0ms preprocess, 171.8ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)


error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1272: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


In [ ]:
results[0].names

{0: 'Corner Back',
 1: 'Fullback',
 2: 'Linebacker',
 3: 'Quarterback',
 4: 'Running Back',
 5: 'Safety',
 6: 'Tight End',
 7: 'WRs',
 8: 'Wide Receiver'}

In [ ]:
#Testing Model
!yolo task=detect mode=predict model=yolov8n_WR/weights/best.pt conf=0.25 source=NewWRData/test/images save=True

Ultralytics YOLOv8.0.196  Python-3.11.0 torch-2.3.0+cpu CPU (Intel Core(TM) i7-9750H 2.60GHz)
Model summary (fused): 168 layers, 3007403 parameters, 0 gradients, 8.1 GFLOPs

image 1/30 C:\Users\RAZEER\Downloads\WR\NewWRData\test\images\AFCC103_png.rf.6c5fc19c43a9ffb71bae0ba33fdea648.jpg: 640x640 3 Corner Backs, 7 Linebackers, 2 Running Backs, 1 Safety, 1 Tight End, 4 Wide Receivers, 185.3ms
image 2/30 C:\Users\RAZEER\Downloads\WR\NewWRData\test\images\AFCC107_png.rf.44b211717aa69451bf6a3e7dfef3198b.jpg: 640x640 4 Corner Backs, 5 Linebackers, 1 Running Back, 4 Safetys, 1 Wide Receiver, 126.0ms
image 3/30 C:\Users\RAZEER\Downloads\WR\NewWRData\test\images\AFCC125_png.rf.6caf6bfd63e18b8beb9434288211b425.jpg: 640x640 4 Corner Backs, 1 Linebacker, 2 Safetys, 4 Wide Receivers, 129.4ms
image 4/30 C:\Users\RAZEER\Downloads\WR\NewWRData\test\images\AFCC130_png.rf.539218f12c1e50c4dce0e1f1b26f9698.jpg: 640x640 1 Corner Back, 1 Linebacker, 5 Safetys, 4 Wide Receivers, 123.7ms
image 5/30 C:\Users\R

In [ ]:
import glob
from IPython.display import Image, display

for image_path in glob.glob('../WR/runs/detect/predict3/*.jpg')[:3]:
      display(Image(filename=image_path, width=600))
      print("\n")